In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install my_krml_25552249==2025.0.6.2


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../data/raw/OnlineNewsPopularity.csv')

In [5]:
df.head(5)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [6]:
df_cleaned = df.copy()

In [7]:
df_cleaned.drop('url', axis=1, inplace=True)

In [8]:
df_cleaned.columns = df_cleaned.columns.str.strip()

In [9]:
from my_krml_25552249.data.sets import pop_target

In [10]:
features, target = pop_target(df_cleaned, 'shares')

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()

In [13]:
features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

In [14]:
from joblib import dump

In [15]:
dump(scaler, '../models/scaler.joblib')

['../models/scaler.joblib']

In [16]:
from my_krml_25552249.data.sets import split_sets_random

In [17]:
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(features, target, test_ratio=0.2)

In [18]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(23786, 59)
(7929, 59)
(7929, 59)


In [19]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(23786,)
(7929,)
(7929,)


In [20]:
from my_krml_25552249.data.sets import save_sets
save_sets(X_train, y_train, X_val, y_val, X_test, y_test, path='../data/processed/')

In [21]:
from sklearn.dummy import DummyRegressor

base_model = DummyRegressor(strategy='mean')
base_model.fit(X_train, y_train)        # fit on training data
y_base = base_model.predict(X_train)    # predict on same training features

from my_krml_25552249.models.performance import print_regressor_scores
print_regressor_scores(y_preds=y_base, y_actuals=y_train, set_name='Training')

RMSE Training: 11854.785261497587
MAE Training: 3191.7838929535715
